# 어린이 보호 구역 데이터

<h1> 어린이 보호 구역 데이터 수집 및 전처리 진행 </h1>

 - 데이터 불러오기
   - 전국 어린이 보호 구역 데이터
 - 데이터 확인 및 전처리
   - 전국 데이터 => 서울 데이터
   - 결측치 확인 => 결측치 처리
 - 데이터 저장하기

## 데이터 불러오기

In [502]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [503]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [504]:
# 전국 어린이 보호 구역 위치
# 18,777개 데이터 / 15개 컬럼

data = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/raw data/서울시 어린이보호구역 위치도.csv', encoding='cp949')

In [505]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18777 entries, 0 to 18776
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시설종류      18777 non-null  object 
 1   대상시설명     18777 non-null  object 
 2   소재지도로명주소  18625 non-null  object 
 3   소재지지번주소   15514 non-null  object 
 4   위도        18769 non-null  float64
 5   경도        18767 non-null  float64
 6   관리기관명     18777 non-null  object 
 7   관할경찰서명    18777 non-null  object 
 8   CCTV설치여부  18777 non-null  object 
 9   CCTV설치대수  13871 non-null  float64
 10  보호구역도로폭   13189 non-null  object 
 11  데이터기준일자   18777 non-null  object 
 12  제공기관코드    18777 non-null  int64  
 13  제공기관명     18777 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 2.0+ MB


## 데이터 확인 및 전처리

In [506]:
# 서울데이터만 필터
# 소재지도로명주소와 소재지지번주소에 결측치가 존재

data.loc[data['제공기관명'].str.contains('서울'), '지역']='서울'

In [507]:
# 지역이 서울인 데이터만 뽑고, 사용할 컬럼(시설종류, 대상시설명, 위도, 경도, 제공기관명)만 골라서 새로운 DataFrame 생성
# CPZ(Children Protection Zone)
CPZ = data[data['지역']=='서울']
col = ['시설종류', '대상시설명', '위도', '경도', '제공기관명']
CPZ = data[data['지역']=='서울'][col]
CPZ.head(5)

,시설종류,대상시설명,위도,경도,제공기관명
114,초등학교,화랑초교,37.626918,127.093803,서울특별시 노원구
115,초등학교,공릉초교,37.618615,127.074631,서울특별시 노원구
116,초등학교,중원초교,37.643983,127.062768,서울특별시 노원구
117,초등학교,원광초교,37.654312,127.074916,서울특별시 노원구
118,학원,국풍2000학원,37.650631,127.076703,서울특별시 노원구


### 데이터 확인

In [508]:
# 항목별 결측치 확인
CPZ.isnull().sum()

시설종류     0
대상시설명    0
위도       4
경도       6
제공기관명    0
dtype: int64

### 데이터 컬럼별 정보

#### 시설종류

초등학교, 유치원, 어린이집, 학원을 제외한 나머지 값은 특수학교로 변환

In [509]:
# 시설종류

CPZ['시설종류'].value_counts()
CPZ['시설종류'].replace({'외국인학교' : '특수학교', '국제학교' : '특수학교', '외국인교육기관' : '특수학교', '대안학교' : '특수학교'}, inplace=True)

#### 대상시설명
1,607개 종류의 대상시설명 존재(1,557개는 1개씩, 50개는 이름이 같은 시설) => 이름만 같고 지역은 다름

#### 위도 & 경도
 - 위도 결측치 4개, 경도 결측치 6개
 - 결측치는 구글 위도경도 검색
 - 아직 완공되지 않은 어린이집 2곳은 데이터 삭제

In [510]:
CPZ[CPZ['위도'].isna()]

,시설종류,대상시설명,위도,경도,제공기관명
13168,유치원,우현유치원,NaN,NaN,서울특별시 강서구
16125,어린이집,신반포센트럴자이어린이집(진행중),NaN,NaN,서울특별시 서초구
16499,초등학교,신자초등학교,NaN,127.073328,서울특별시 광진구
17684,어린이집,신반포(보육지원센터),NaN,NaN,서울특별시 서초구


In [511]:
CPZ[CPZ['경도'].isna()]

,시설종류,대상시설명,위도,경도,제공기관명
10669,유치원,신자병설유치원,37.530017,NaN,서울특별시 광진구
13168,유치원,우현유치원,NaN,NaN,서울특별시 강서구
16125,어린이집,신반포센트럴자이어린이집(진행중),NaN,NaN,서울특별시 서초구
16533,초등학교,반포,37.502485,NaN,서울특별시 서초구
16534,유치원,반포초교병설,37.502485,NaN,서울특별시 서초구
17684,어린이집,신반포(보육지원센터),NaN,NaN,서울특별시 서초구


In [512]:
# 신자병설유치원 경도: 127.075438
# 우현유치원 위도: 37.557521, 경도: 126.838107
# 신반포센트럴자이어린이집 ?
# 반포초등학교 위도: 37.502987, 경도: 126.991036
# 반포초교병설유치원 위도: 37.503333, 경도: 126.990979
# 신반포어린이집 ?
# 신자초등학교 위도: 37.530315

In [513]:
CPZ.loc[10669,'경도'] = 127.075438
CPZ.loc[13168,'경도'] = 126.838107
CPZ.loc[16533,'경도'] = 126.991036
CPZ.loc[16534,'경도'] = 126.990979
CPZ.loc[13168,'위도'] = 37.557521
CPZ.loc[16533,'위도'] = 37.502987
CPZ.loc[16534,'위도'] = 37.503333
CPZ.loc[16499,'위도']= 37.530315

In [514]:
# 결측치 값 삭제 (503개)
CPZ = CPZ.dropna()

In [515]:
CPZ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1669 entries, 114 to 18626
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시설종류    1669 non-null   object 
 1   대상시설명   1669 non-null   object 
 2   위도      1669 non-null   float64
 3   경도      1669 non-null   float64
 4   제공기관명   1669 non-null   object 
dtypes: float64(2), object(3)
memory usage: 78.2+ KB


In [516]:
# 구만 구분하여 저장

CPZ['행정구역'] = CPZ['제공기관명'].str.strip("서울특별시").str.strip()
len(CPZ['행정구역'].unique())
CPZ.drop('제공기관명', axis=1, inplace=True)
CPZ = CPZ[['행정구역', '시설종류', '대상시설명', '위도', '경도']]
array = np.arange(len(CPZ['행정구역']))
CPZ.index=array

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [517]:
CPZ

,행정구역,시설종류,대상시설명,위도,경도
0,노원구,초등학교,화랑초교,37.626918,127.093803
1,노원구,초등학교,공릉초교,37.618615,127.074631
2,노원구,초등학교,중원초교,37.643983,127.062768
3,노원구,초등학교,원광초교,37.654312,127.074916
4,노원구,학원,국풍2000학원,37.650631,127.076703
...,...,...,...,...,...
1664,마포구,초등학교,서교초교,37.554889,126.924378
1665,마포구,초등학교,성산초교,37.553410,126.910949
1666,마포구,초등학교,신북초교,37.571104,126.905363
1667,마포구,초등학교,한서초교,37.553648,126.949779


In [518]:
# # 범위로 되어있는 값은 중간값(평균값)을 구해서 변환
# dict_word = {',':'~', '-':'~', '`':'~'}

# for i in range(len(CPZ['행정구역'])):
#   CPZ['보호구역도로폭'][i] = CPZ['보호구역도로폭'][i].translate(CPZ['보호구역도로폭'][i].maketrans(dict_word))
#   if '~' in CPZ['보호구역도로폭'][i]:
#     CPZ['보호구역도로폭'][i] = int(float(CPZ['보호구역도로폭'][i].split('~')[0])+float(CPZ['보호구역도로폭'][i].split('~')[1])/2)
#   else:
#     CPZ['보호구역도로폭'][i] = float(CPZ['보호구역도로폭'][i])

In [519]:
# 결측치 제거
CPZ = CPZ.dropna()

In [520]:
CPZ.reset_index(inplace=True)

In [521]:
CPZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   1669 non-null   int64  
 1   행정구역    1669 non-null   object 
 2   시설종류    1669 non-null   object 
 3   대상시설명   1669 non-null   object 
 4   위도      1669 non-null   float64
 5   경도      1669 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 78.4+ KB


## 데이터 저장

기존 1,671개의 데이터에서 위도와 경도의 결측치가 존재하는 데이터 2개를 삭제하여 총 1,669 데이터 전처리

In [522]:
CPZ.to_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/서울시 어린이보호구역.csv', index=False)

# 서울시 어린이보호구역.csv
 - 서울에 있는 어린이 보호구역 1,669개 데이터
 - 5개 컬럼

| 컬럼 | 개수 | 타입 | 데이터 예시 |
|:------:|:------:|:------:|:-------------|
|시설종류|1669|object|총 9개(초등학교, 유치원, 어린이집, 학원, 특수학교, 외국인학교, 국제학교, 외국인교육기관, 대안학교)|
|대상시설명|16769|object|총 1,607개|
|구명|1669|object|서울의 각 구 별로 25개|
|경도|1669|float64|경도: xxx.xxxxxx|
|위도|1669|float64|위도: xx.xxxxxx|

